In [1]:
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense,LSTM,Embedding
from keras.preprocessing.sequence import pad_sequences
filepath=[r"B:\project\human\human_chunk_10.csv",r"B:\project\ai\ai_chunk_10.csv"]
dfs = [pd.read_csv(filepath) for filepath in filepath]
df = pd.concat(dfs, ignore_index=True)
print(df.shape)


(30580, 2)


In [2]:
essay=df['text'].tolist()
from transformers import LongformerTokenizer
tokenizer=LongformerTokenizer.from_pretrained('allenai/longformer-base-4096')
essay=df['text'].tolist()
tokens=tokenizer(essay,return_tensors='pt',padding=True,truncation=True)

In [3]:
x=tokens['input_ids']
y=df['generated']
xtrain,xtest,ytrain,ytest=train_test_split(x,y,test_size=0.2,random_state=42)
# input_ids_size = tokens['input_ids'].size()
# attention_mask_size = tokens['attention_mask'].size()
# print("Input IDs size:", input_ids_size)
# print("Attention Mask size:", attention_mask_size)
# # Assuming 'tokens' is the result of tokenizer
# unique_tokens = set(token.item() for seq in tokens['input_ids'] for token in seq)
# vocabulary_size = len(unique_tokens)

# print("Vocabulary Size:", vocabulary_size)
# # Assuming 'input_ids' is the first element in xtrain
# max_sequence_length_x = max(len(seq) for seq in xtrain)

# # Print or use the max_sequence_length_x in your pad_sequences and Embedding layer
# print("Max Sequence Length:", max_sequence_length_x)


In [4]:
from tensorflow.keras.layers import Input, Embedding, Conv1D, GlobalMaxPooling1D, Dense, Dropout
from tensorflow.keras.models import Model
xtrain_np=xtrain.numpy()
input_length = None
input_layer = Input(shape=(input_length,))
embedding_layer = Embedding(input_dim=51000, output_dim=128, input_length=input_length)(input_layer)
conv1d_layer = Conv1D(128, 5, activation='relu')(embedding_layer)
maxpool_layer = GlobalMaxPooling1D()(conv1d_layer)
dense_layer = Dense(64, activation='relu')(maxpool_layer)
dropout_layer = Dropout(0.5)(dense_layer)
output_layer = Dense(1, activation='sigmoid')(dropout_layer)

model = Model(inputs=input_layer, outputs=output_layer)

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Assuming xtrain_np and ytrain are NumPy arrays
model.fit(xtrain_np, ytrain, epochs=5, batch_size=256)




Epoch 1/5


96/96 [==============================] - 84s 860ms/step - loss: 0.3517 - accuracy: 0.8627
Epoch 2/5
96/96 [==============================] - 91s 954ms/step - loss: 0.0422 - accuracy: 0.9881
Epoch 3/5
96/96 [==============================] - 91s 944ms/step - loss: 0.0146 - accuracy: 0.9967
Epoch 4/5
96/96 [==============================] - 93s 966ms/step - loss: 0.0043 - accuracy: 0.9996
Epoch 5/5
96/96 [==============================] - 90s 940ms/step - loss: 0.0019 - accuracy: 0.9999


In [5]:
xtest_np=xtest.numpy()
model.evaluate(xtest_np,ytest)

192/192 [==============================] - 5s 27ms/step - loss: 0.0309 - accuracy: 0.9895


[0.030878392979502678, 0.9895356297492981]

In [6]:
model.save("dmodel10.keras")
model.save_weights('dmodel10_weights.keras')

In [7]:
from keras.models import load_model
from keras.preprocessing.sequence import pad_sequences
ml = load_model("dmodel10.keras")
# xnew="Studying Science and History at Generic_School offers two very different experiences each with its own advantages Science is a constantly evolving field with new advances and techniques being discovered daily The school provides expert guidance and modern equipment making it an ideal place to explore the complex theories and technologies of the subject In contrast History offers an opportunity to delve into the past and investigate the origins of our civilization and culture Through lectures and field trips Generic_School s expert faculty provides students with a solid foundation of knowledge in the subject while also allowing them to study and discuss unique interpretations of traditional narratives Both Science and History have their advantages but in the end it comes down to each individual student s interests and goals If you are passionate about understanding the complexities of our world then studying Science is the perfect choice If on the other hand you enjoy delving into the past and uncovering the stories of our ancestors then History is the subject for you Whichever path you choose to take the experienced faculty and resources at Generic_School will provide the guidance you need to succeed and make the most of your studies"
# xnew = "hi my name is yogeskumaran i am from madurai, where are you from"
# Tokenize the input text
# xnew="Cars. Cars have been around since they became famous in the 1900s, when Henry Ford created and built the first ModelT. Cars have played a major role in our every day lives since then. But now, people are starting to question if limiting car usage would be a good thing. To me, limiting the use of cars might be a good thing to do.In like matter of this, article,In German Suburb, Life Goes On Without Cars,by Elizabeth Rosenthal states, how automobiles are the linchpin of suburbs, where middle class families from either Shanghai or Chicago tend to make their homes. Experts say how this is a huge impediment to current efforts to reduce greenhouse gas emissions from tailpipe. Passenger cars are responsible for 12 percent of greenhouse gas emissions in Europe...and up to 50 percent in some carintensive areas in the United States. Cars are the main reason for the greenhouse gas emissions because of a lot of people driving them around all the time getting where they need to go. Article, Paris bans driving due to smog, by Robert Duffer says, how Paris, after days of nearrecord pollution, enforced a partial driving ban to clear the air of the global city. It also says, how on Monday, motorist with evennumbered license plates were ordered to leave their cars at home or be fined a 22euro fine 31. The same order would be applied to oddnumbered plates the following day. Cars are the reason for polluting entire cities like Paris. This shows how bad cars can be because, of all the pollution that they can cause to an entire city.Likewise, in the article,Carfree day is spinning into a big hit in Bogota,by Andrew Selsky says, how programs thats set to spread to other countries, millions of Columbians hiked, biked, skated, or took the bus to work during a carfree day, leaving streets of this capital city eerily devoid of traffic jams. It was the third straight year cars have been banned with only buses and taxis permitted for the Day Without Cars in the capital city of 10 million. People like the idea of having carfree days because, it allows them to lesson the pollution that cars put out of their exhaust from people driving all the time. The article also tells how parks and sports centers have bustled throughout the city uneven, pitted sidewalks have been replaced by broad, smooth sidewalks rushhour restrictions have dramatically cut traffic and new restaurants and upscale shopping districts have cropped up. Having no cars has been good for the country of Columbia because, it has aloud them to repair things that have needed repairs for a long time, traffic jams have gone down, and restaurants and shopping districts have popped up, all due to the fact of having less cars around.In conclusion, the use of less cars and having carfree days, have had a big impact on the environment of cities because, it is cutting down the air pollution that the cars have majorly polluted, it has aloud countries like Columbia to repair sidewalks, and cut down traffic jams. Limiting the use of cars would be a good thing for America. So we should limit the use of cars by maybe riding a bike, or maybe walking somewhere that isnt that far from you and doesnt need the use of a car to get you there. To me, limiting the use of cars might be a good thing to do."
xnew="There are alot reasons to keep our the despised method of choosing the President The First reason is because Certainty of Outcome in the text its states that The reason is that the winning candidate s share of the Electoral College invariably exceeds his share of the popular vote Another example from the text is Obama he recieved 61 10 percent of the electoral vote compared to only 51 3 percent of the popular votes cast for him and Romney because all of the states award electoral votes on a winner take all basis Another reason is that the Electiral College requires a presidential candidate to have trans regional appeal in that case no region has enough electoral votes to elect a president a solid regional favorite such as Romney was in the South has no reason to campaign in those states because of the poor economy and because its not a wealthy state he wont be able to campaign heavily So he left with no votes and would have to find his Electoral College votes in a more suitable area In the text its states that It is a Desriable result because a candidate with regional appeal is unlikely to be a successful president Also the Electoral College restores some of the weight in the political balance the large states by population lose by virtue of the mal apportionment Senate decreed in the Constitution My proof is in the text The popular vote was very close in Florida in 2012 nevertheless Obama who won the that vote got 29 electoral votes A same victory by the same margin in Wynoming would net the winner only 3 electoral votes So other things equal a large state gets more attention compared to a small state And Finally the Electoral College avoids the problemof elections in which no candidate recieves a majority of the votes cast For example Richard Nixon in 68 and Bill Clinton in 92 both had on 43 percent plurality of the popular votes while winning a majority in the in the Electoral College In Conclusion I think that the we should keep Electoral Colleges and changing to election by popular voting to vote for a President"
xnew_tokens = tokenizer.encode(xnew,truncation=True,padding=True,return_tensors="np")

max_sequence_length=1823
xnew_padded = pad_sequences(xnew_tokens, maxlen=max_sequence_length, padding='post', truncating='post')
prediction = ml.predict(xnew_padded)
print(prediction)
# Set a threshold for classification
threshold = 0.5
prediction_value = prediction[0, 0]

# Check the prediction and print the result
if prediction_value >= threshold:
    print("1: It is an AI-generated text")
else:
    print("0: It is Human-generated text")


1/1 [==============================] - 0s 133ms/step
[[0.00024383]]
0: It is Human-generated text


In [8]:
x=tokens['input_ids']
y=df['generated']
xtrain,xtest,ytrain,ytest=train_test_split(x,y,test_size=0.2,random_state=42)
xtrain_np=xtrain.numpy()
xtest_np=xtest.numpy()
xtrain_np10=xtrain_np
xtest_np10=xtest_np
import numpy as np
xtrain_np10 = np.array(xtrain_np10)
xtest_np10=np.array(xtest_np10)
np.savetxt('B:/project/test/xtest_np10.csv', xtest_np10, delimiter=',')
np.savetxt('B:/project/train/xtrain_np10.csv', xtrain_np10, delimiter=',')
ytest.to_csv("B:/project/ytest/ytest10.csv")

In [9]:
x=tokens['input_ids']
y=df['generated']
xtrain,xtest,ytrain,ytest=train_test_split(x,y,test_size=0.2,random_state=42)
ytrain.to_csv("B:/project/ytrain/ytrain10.csv")
